In [1]:
import pandas as pd
import numpy as np
import os

import progressbar
from time import sleep

import warnings
warnings.filterwarnings('ignore')

# настройка формата числовых значений без плавающей запятой
# pd.options.display.float_format = '{:.0f}'.format
pd.reset_option('all')

# настройка полного отображения столбца
pd.set_option('display.max_colwidth', -1)

# объявление переменной, которая содержит файл xlsx
file = "M:\\Контакт-центр\\Аналитический отдел\\Джумагалиев\\Загрузка номеров\\МИР\\Август\\Результат МИР Август 2 партия.xlsx"
#file = "M:\\Контакт-центр\\Аналитический отдел\\Джумагалиев\\Загрузка номеров\\ТНТ\\Копия ТНТ 1.xlsx"
xlsx = pd.ExcelFile(file)

# парсим рабочий лист в переменную
df = xlsx.parse(xlsx.sheet_names[0])
df = df.convert_dtypes()
#df['Номер паспорта'] = df['Номер паспорта'].astype('str')
df['Дата рождения'] = df['Дата рождения'].astype('str')

# замена NaN-значений на пустые значения
df = df.fillna(value='')
#df['Номер паспорта'] = df['Номер паспорта'].replace('<NA>', '')

# приведение значений ФИО к виду "все первые буквы - прописные"
df['Фамилия'] = df['Фамилия'].str.title()
df['Имя'] = df['Имя'].str.title()
df['Отчество'] = df['Отчество'].str.title()

# СЦЕПИТЬ ФИО и ДР
df['КЛ1'] = df['Фамилия']+ ' ' + df['Имя'] + ' ' + df['Отчество'] + ', ' + df['Дата рождения'].astype(str)

# удаляем ненужные столбцы
df.drop(['Фамилия', 'Имя', 'Отчество', 'Дата рождения'], axis=1, inplace=True)

# меняем порядок столбцов
column_names = ["ClientID", "КЛ1", "Номер телефона", "Дата актуальности"]
df = df.reindex(columns=column_names)

df['ID_KL'] = df['ClientID'].astype(str) + ' ' + df['КЛ1']
df['ID_KL'].unique()

df['ClientID'].unique()

bar = progressbar.ProgressBar(maxval=len(df['ClientID'].unique()), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

# запись в датафрейм 2 актуальных номеров каждого ФЛ (удаление 3+ номеров)
df_2num = df.iloc[0:0]
for num_index, num_item in enumerate(df['ID_KL'].unique()):
    df_2num = df_2num.append((df[df['ID_KL'] == num_item].head(2)), ignore_index=True)

# формируем столбец "СкипКБРС_номертелефона"
df_2num['Номер телефона'] = df_2num['Номер телефона'].apply(lambda x: "СкипСОЛ_" + str(x))

# создание итогового датафрейма
df_OUT = pd.DataFrame({'Идентификатор': pd.Series(dtype='int'),
                        'СД': pd.Series(dtype='str')})

# генерируем столбцы по максимальному числу Физических Лиц
for i in range(df_2num.groupby(['ClientID'])['ID_KL'].nunique().sort_values(ascending=False).values[0]):
    df_OUT['кл'+str(i+1)] = ''
    df_OUT['тел'+str(i+1)] = ''
    
# записать
bar.start()
for ID_index, ID_item in enumerate(df_2num['ClientID'].unique()):
    df_OUT = df_OUT.append({'Идентификатор': ID_item}, ignore_index=True)
    mask_ID = df_2num.query('ClientID ==' + str(ID_item), engine='python')
    for KL_index, KL_item in enumerate(mask_ID['КЛ1'].unique()):
        df_OUT.at[ID_index, str("кл"+str(KL_index+1))] = KL_item
        mask_KL = df_2num.loc[df_2num['КЛ1'] == KL_item]
        for TEL_index, TEL_item in enumerate(mask_KL['Номер телефона'].unique()):
            df_OUT.at[ID_index, str("тел"+str(KL_index+1))] = str(str(df_OUT.at[ID_index, str("тел"+str(KL_index+1))]) + str(TEL_item) + ";")
    bar.update(ID_index+1)
    sleep(0.1)
bar.finish()
df_OUT = df_OUT.convert_dtypes()
df_OUT.replace(to_replace=r'nan', value='', regex=True, inplace=True)
df_OUT

df_OUT.to_excel("E:\\Users\\e.dzhumagaliev\\Downloads\\out_phone_num.xlsx")

[========================================================================] 100%
